In [1]:
import pandas as pd
import os
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from pathlib import Path
import glob

# Date to split the files result

In [2]:
date_list = [['5-Aug-2022','12-Aug-2022', '19-Aug-2022', '26-Aug-2022', '2-Sep-2022', '9-Sep-2022',
             '16-Sep-2022', '23-Sep-2022', '28-Sep-2022', '30-Sep-2022'], ['6-Oct-2022'], ['7-Oct-2022'], 
             ['14-Oct-2022'],['21-Oct-2022'], ['28-Oct-2022'], ['4-Nov-2022']]
type(date_list)

list

# Extracting the dynamic file and filtering the ongoing out for matching

In [3]:
input_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Target/'
output_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Result/'

In [4]:
input_loc_d = input_loc+"Dynamic Source Latest/"

In [5]:
os.chdir(input_loc+"Dynamic Source Latest/")
filelist = glob.glob(input_loc_d+'**/*.csv', recursive = True)
filelist

['/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Target/Dynamic Source Latest/Dynamic_source.csv']

In [6]:
dynamic_source = pd.DataFrame()

for files in filelist:
    if files.endswith(".csv"):
        df = pd.read_csv(files)
        dynamic_source = dynamic_source.append(df)

# unsuccessful_revisit.to_excel(output_loc+"unsuccessful_revisit.xlsx", index = False)

In [7]:
dynamic_source.shape

(1991413, 19)

In [8]:
dynamic_source.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,Ingestion Date,RWM Remarks,Revisit Expire Date,s no,RWM Revisit Status,RWM Revisit Date,Validity,Pincode,Phone Number,City,Cluster,Failed Upload RWM MID,csv Google MID,Google csv Visit Date,csv Validity
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,5-Aug-2022,Success,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dynamic_source.sort_values(by=['s no'],inplace=True)

In [10]:
dynamic_source['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022'],
      dtype=object)

# Reading the mid_match

In [11]:
mid_match = pd.read_csv(input_loc+"q1 + q2 Source MID Match/mid_match.csv")

In [12]:
mid_match.head(1)

,Merchant Ext ID,Primary_key,s no,Name,Phone Number,Address,Pincode,Latitude,Longitude,City,Polygon Name,Created By ID,Last Transaction date,Gpay OB E-mail,segments,Source_Tags,_merge,Cluster
0,BCR2DN6TTOL5HUDM,1,1718726,Sharda Healthcare,9.811409e+09,Main Teekli Road Near Police Station Badshahpu...,122001.0,28.389893,77.04838,Gurugram,122001,58517.0,09 March 2022,SahirK171@gmail.com,active,q2,both,Delhi


In [13]:
mid_match_f = mid_match[['s no', 'Pincode','Phone Number', 'City', 'Cluster', 'segments']].rename(
    columns={'segments':'Objectives_2', 'Pincode':'Pincode_2', 'Phone Number':'Phone Number_2', 'City':'City_2',
            'Cluster':'Cluster_2', })

# Extracting the Google Revisit and RWM revisit file to match MID to the Dynamic Source MID

## Google Revisit

### Create Variables for Input and Output location

In [14]:
input_loc_google_r = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Target/Google Revisit File/2022/08/"

### Create Variables for list of input files for Google Revisit

In [15]:
filelist = os.listdir(input_loc_google_r)
filelist

["09 Nov'22 Revisit - Google Revisit.csv",
 ".~lock.20 Sep'22 Revisit - Google Revisit.csv#",
 "06 Nov'22 Revisit - Google Revisit.csv",
 "04 Nov'22 Revisit - Google Revisit.csv",
 "08 Nov'22 Revisit - Google Revisit.csv",
 "05 Nov'22 Revisit - Google Revisit.csv",
 "07 Nov'22 Revisit - Google Revisit.csv"]

#### Consolidate the file using for loop

In [16]:
google_revisit = pd.DataFrame()

for files in filelist:
    if files.endswith(".csv"):
        df = pd.read_csv(input_loc_google_r + files)
        google_revisit = google_revisit.append(df)

google_revisit.to_csv(output_loc+"google_revisit.csv", index = False)

## RWM Revisit

#### Create Variables for Input and Output location¶

In [17]:
input_loc_rwm_r = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Target/Revisit Merchant Master Records/2022/08/"

#### Create Variables for list of input files for Google Revisit¶ 

In [18]:
os.chdir(input_loc_rwm_r)
filelist = glob.glob(input_loc_rwm_r+'/**/*.xlsx')

#for f in filelist:
#    print(f)

#### Consolidate the file using for loop

In [19]:
rwm_revisit = pd.DataFrame()

for files in filelist:
    if files.endswith(".xlsx"):
        df = pd.read_excel(files)
        rwm_revisit = rwm_revisit.append(df)

rwm_revisit.to_excel(output_loc+"rwm_revisit.xlsx", index = False)

# Filtering Column from the revisit and changing datetime format

In [20]:
rwm_revisit_f = rwm_revisit[['Merchant ID', 'Owner Mobile No.', 'Revisit Date', 'Remarks', 'QR Code String']]
google_revisit_f = google_revisit[['MerchantExternalId', 'StorePhoneNumber', 'TaskSubmissionTimestamp','Validity']]

In [21]:
rwm_revisit_f['Revisit Date'] = pd.to_datetime(rwm_revisit_f['Revisit Date'])
google_revisit_f['TaskSubmissionTimestamp'] = pd.to_datetime(google_revisit_f['TaskSubmissionTimestamp'])

In [22]:
rwm_revisit_f['Revisit Date'] = rwm_revisit_f['Revisit Date'].dt.strftime('%-d-%b-%Y')
google_revisit_f['TaskSubmissionTimestamp'] = google_revisit_f['TaskSubmissionTimestamp'].dt.strftime('%-d-%b-%Y')

In [23]:
google_revisit_f['TaskSubmissionTimestamp'].replace('30-Dec-1899', np.nan, inplace=True)

In [24]:
google_revisit_f['TaskSubmissionTimestamp'].fillna(method='ffill', inplace=True)

In [25]:
google_revisit_f['TaskSubmissionTimestamp'].fillna(method='bfill', inplace=True)

# Filtering revisit done from RWM

In [26]:
rwm_revisit_f2 = rwm_revisit_f[rwm_revisit_f['Remarks'].isin(['Revisit Done'])]

In [27]:
rwm_revisit_split = rwm_revisit_f2['QR Code String'].str.split('=', expand = True).rename(
    columns={5:'MerchantId'}).filter(['MerchantId'])

In [28]:
rwm_revisit_concat = pd.concat([rwm_revisit_f2, rwm_revisit_split], axis = 1, ignore_index = True)

In [29]:
rwm_revisit_concat_f = rwm_revisit_concat[[0,1,2,3,5]]

In [30]:
rwm_revisit_concat_f.head(1)

,0,1,2,3,5
322,8291321,9350628886,6-Nov-2022,Revisit Done,BCR2DN4TQC54X4BD


In [31]:
rwm_revisit_concat_f.rename(
    columns={0:'Merchant ID',1:'Owner Mobile No.', 2:'Revisit Date',3:'Remarks',5:'MerchantId'}, inplace=True)

In [32]:
rwm_revisit_concat_f2 = rwm_revisit_concat_f[['MerchantId', 'Revisit Date', 'Remarks']]

In [33]:
rwm_revisit_concat_f2.head(1)

,MerchantId,Revisit Date,Remarks
322,BCR2DN4TQC54X4BD,6-Nov-2022,Revisit Done


# Filtering columns in Google Revisit

In [34]:
google_revisit_f.head(1)

,MerchantExternalId,StorePhoneNumber,TaskSubmissionTimestamp,Validity
0,BCR2DN4T7C5JLWAV,8866609035,9-Nov-2022,INVALID


In [35]:
google_revisit_f2 = google_revisit_f[['MerchantExternalId', 'TaskSubmissionTimestamp', 'Validity']].rename(
    columns={'MerchantExternalId':'MerchantId', 'TaskSubmissionTimestamp':'New_Revisit_Date_Google', 
             "Validity":'Google_Validity'})

In [36]:
google_revisit_f2['csv Google MID'] = google_revisit_f2['MerchantId']

# Filtering out the Dynamic Source

In [37]:
dynamic_source['Primary_key'] = range(1,(1+len(dynamic_source)))

In [38]:
dynamic_source_final = dynamic_source

In [39]:
dynamic_source = dynamic_source[dynamic_source['RWM Remarks'].isin(['Success'])]

In [40]:
dynamic_source_f = dynamic_source.drop_duplicates(subset=['MerchantId'], keep='last')

In [41]:
dynamic_source_f2 = dynamic_source_f[['MerchantId', 'IngestionStatus', 'Revisit Expire Date', 'Primary_key']]

In [42]:
dynamic_source_f2.head()

,MerchantId,IngestionStatus,Revisit Expire Date,Primary_key
0,BCR2DN4TSCO3DNDR,ADD,NaN,1
2,BCR2DN6T2OKJNTKZ,EDIT,NaN,3
3,BCR2DN6TUXLJ3PTX,EDIT,NaN,4
5,BCR2DN6TWO2ZPXL5,EDIT,NaN,6
6,BCR2DN6T56P37XKS,EDIT,NaN,7


# Matching with RWM Revisit Done

In [43]:
rwm_revisit_concat_f2.sort_values(by=['Revisit Date'], inplace=True)

In [44]:
dynamic_source_f2_merge_rwm = dynamic_source_f2.merge(rwm_revisit_concat_f2, on="MerchantId", how='left')

In [45]:
dynamic_source_f2_merge_rwm.drop_duplicates(subset=['Primary_key'], inplace=True)

In [46]:
dynamic_source_f2_merge_rwm.shape

(1284684, 6)

# Matching with Google Revisit

In [47]:
google_revisit_f2.fillna(method="bfill", inplace=True)

In [48]:
google_revisit_f2.sort_values(by=['New_Revisit_Date_Google'], inplace=True)

In [49]:
dynamic_source_f2_merge_rwm_google = dynamic_source_f2_merge_rwm.merge(
    google_revisit_f2, on='MerchantId', how='left')

In [50]:
dynamic_source_f2_merge_rwm_google.drop_duplicates(subset=['Primary_key'], inplace=True)

In [51]:
dynamic_source_f2_merge_rwm_google.head()

,MerchantId,IngestionStatus,Revisit Expire Date,Primary_key,Revisit Date,Remarks,New_Revisit_Date_Google,Google_Validity,csv Google MID
0,BCR2DN4TSCO3DNDR,ADD,NaN,1,NaN,NaN,NaN,NaN,NaN
1,BCR2DN6T2OKJNTKZ,EDIT,NaN,3,NaN,NaN,NaN,NaN,NaN
2,BCR2DN6TUXLJ3PTX,EDIT,NaN,4,NaN,NaN,NaN,NaN,NaN
3,BCR2DN6TWO2ZPXL5,EDIT,NaN,6,NaN,NaN,NaN,NaN,NaN
4,BCR2DN6T56P37XKS,EDIT,NaN,7,NaN,NaN,NaN,NaN,NaN


In [52]:
dynamic_source_f2_merge_rwm_google.rename(columns={'Revisit Date':'New_Revisit_Date_RWM', 'Remarks':'Remarks_RWM',
                                                   'Revisit Date':'Revisit_Date_RWM'}, inplace=True)

In [53]:
dynamic_source_f2_merge_rwm_google.head()

,MerchantId,IngestionStatus,Revisit Expire Date,Primary_key,Revisit_Date_RWM,Remarks_RWM,New_Revisit_Date_Google,Google_Validity,csv Google MID
0,BCR2DN4TSCO3DNDR,ADD,NaN,1,NaN,NaN,NaN,NaN,NaN
1,BCR2DN6T2OKJNTKZ,EDIT,NaN,3,NaN,NaN,NaN,NaN,NaN
2,BCR2DN6TUXLJ3PTX,EDIT,NaN,4,NaN,NaN,NaN,NaN,NaN
3,BCR2DN6TWO2ZPXL5,EDIT,NaN,6,NaN,NaN,NaN,NaN,NaN
4,BCR2DN6T56P37XKS,EDIT,NaN,7,NaN,NaN,NaN,NaN,NaN


# Filtering Out Delete and Checking it's Validity

In [54]:
dynamic_source_f2_merge_rwm_google_f_del = dynamic_source_f2_merge_rwm_google[dynamic_source_f2_merge_rwm_google[
    'IngestionStatus'].isin(['DELETE'])]

In [55]:
dynamic_source_f2_merge_rwm_google_f_del['Revisit Expire Date'] = pd.to_datetime(
    dynamic_source_f2_merge_rwm_google_f_del['Revisit Expire Date'])
dynamic_source_f2_merge_rwm_google_f_del['Revisit_Date_RWM'] = pd.to_datetime(
    dynamic_source_f2_merge_rwm_google_f_del['Revisit_Date_RWM'])

In [56]:
dynamic_source_f2_merge_rwm_google_f_del['Revisit Expire Date'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Revisit Expire Date'].dt.strftime('%Y/%-m/%-d')
dynamic_source_f2_merge_rwm_google_f_del['Revisit_Date_RWM'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Revisit_Date_RWM'].dt.strftime('%Y/%-m/%-d')

In [57]:
dynamic_source_f2_merge_rwm_google_f_del['Check_Revisit_Validity'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Revisit Expire Date'] <= dynamic_source_f2_merge_rwm_google_f_del['Revisit_Date_RWM']

In [58]:
dynamic_source_f2_merge_rwm_google_f_del['Check_Revisit_Validity'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Check_Revisit_Validity'].map({True:"VALID", False:"INVALID"})

In [59]:
dynamic_source_f2_merge_rwm_google_f_del.head(1)

,MerchantId,IngestionStatus,Revisit Expire Date,Primary_key,Revisit_Date_RWM,Remarks_RWM,New_Revisit_Date_Google,Google_Validity,csv Google MID,Check_Revisit_Validity
29,BCR2DN6TZO66PUCX,DELETE,2022/8/5,52,NaN,NaN,NaN,NaN,NaN,INVALID


In [60]:
dynamic_source_f2_merge_rwm_google_f_del['Check blank_RWM_Revisit'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Revisit_Date_RWM'].isna()

In [61]:
dynamic_source_f2_merge_rwm_google_f_del['Check blank_RWM_Revisit'] = dynamic_source_f2_merge_rwm_google_f_del[
    'Check blank_RWM_Revisit'].map({True:True, False:np.nan})

In [62]:
dynamic_source_f2_merge_rwm_google_f_del["Check blank_RWM_Revisit"].unique()

array([True, nan], dtype=object)

In [63]:
dynamic_source_f2_merge_rwm_google_f_del["Check blank_RWM_Revisit"].fillna(dynamic_source_f2_merge_rwm_google_f_del[
    'Check_Revisit_Validity'], inplace=True)

In [64]:
dynamic_source_f2_merge_rwm_google_f_del['Check blank_RWM_Revisit'].replace(True, np.nan, inplace=True)

In [65]:
dynamic_source_f2_merge_rwm_google_f_del.rename(
    columns={'Check_Revisit_Validity':'Check blank_RWM_Revisit', 
             'Check blank_RWM_Revisit':'Check_Revisit_Validity'}, inplace=True)

# Merging with the Dynamic Source

In [66]:
dynamic_source_f2_merge_rwm_google_f2_del = dynamic_source_f2_merge_rwm_google_f_del[['Primary_key', 
                                                                                      'Check_Revisit_Validity']]

In [67]:
dynamic_source_f2_merge2_rwm_google_del = dynamic_source_f2_merge_rwm_google.merge(
    dynamic_source_f2_merge_rwm_google_f2_del, on='Primary_key', how='left')

In [68]:
dynamic_source_f2_merge2_rwm_google_del.rename(
    columns={'Check_Revisit_Validity':'RWM_Revisit_Validity'}, inplace=True)

In [69]:
dynamic_source_f3_merge2_rwm_google_del = dynamic_source_f2_merge2_rwm_google_del[['Primary_key', 
    'Revisit_Date_RWM', 'Remarks_RWM', 'New_Revisit_Date_Google', 'Google_Validity', 'csv Google MID',
                                                            'RWM_Revisit_Validity']]

In [70]:
dynamic_source_merge = dynamic_source_final.merge(
    dynamic_source_f3_merge2_rwm_google_del, on='Primary_key', how='left')

In [71]:
dynamic_source_merge.drop_duplicates(subset=['Primary_key'], inplace=True)

In [72]:
dynamic_source_merge.head()

,MerchantId,Objectives,IngestionStatus,Report Date,Ingestion Date,RWM Remarks,Revisit Expire Date,s no,RWM Revisit Status,RWM Revisit Date,...,csv Google MID_x,Google csv Visit Date,csv Validity,Primary_key,Revisit_Date_RWM,Remarks_RWM,New_Revisit_Date_Google,Google_Validity,csv Google MID_y,RWM_Revisit_Validity
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,5-Aug-2022,Success,NaN,1,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
1,BCR2DN4TWDVZF7LF,checkin-nob,ADD,5-Aug-2022,NaN,Merchant Ext.ID is not present in Merchant Mas...,NaN,2,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
2,BCR2DN6T2OKJNTKZ,inactive | stuck_settlements,EDIT,5-Aug-2022,5-Aug-2022,Success,NaN,3,Revisit Done,23-Sep-2022,...,BCR2DN6T2OKJNTKZ,23-Sep-2022,VALID,3,NaN,NaN,NaN,NaN,NaN,NaN
3,BCR2DN6TUXLJ3PTX,support | stuck_settlements,EDIT,5-Aug-2022,5-Aug-2022,Success,NaN,4,NaN,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
4,BCR2DN6TUW2MV52D,stuck_settlements,ADD,5-Aug-2022,NaN,Merchant Ext.ID is not present in Merchant Mas...,NaN,5,NaN,NaN,...,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN


# Filling the blank in Original Dynamic Source

In [73]:
dynamic_source_merge['RWM Revisit Status'].fillna(dynamic_source_merge['Remarks_RWM'], inplace=True)

In [74]:
dynamic_source_merge['RWM Revisit Date'].fillna(dynamic_source_merge['Revisit_Date_RWM'], inplace=True)

In [75]:
dynamic_source_merge['Validity'].fillna(dynamic_source_merge['RWM_Revisit_Validity'], inplace=True)

In [76]:
dynamic_source_merge['csv Google MID_x'].fillna(dynamic_source_merge['csv Google MID_y'], inplace=True)

In [77]:
dynamic_source_merge['Google csv Visit Date'].fillna(dynamic_source_merge['New_Revisit_Date_Google'], inplace=True)

In [78]:
dynamic_source_merge['csv Validity'].fillna(dynamic_source_merge['Google_Validity'], inplace=True)

# Filling Validity Except for Delete

In [79]:
dynamic_source_merge_f2 = dynamic_source_merge[['MerchantId', 'Primary_key', 'IngestionStatus', 
                                                'RWM Revisit Date']]

In [80]:
dynamic_source_merge_f3 = dynamic_source_merge_f2[~dynamic_source_merge_f2['IngestionStatus'].isin(['DELETE'])]

In [81]:
dynamic_source_merge_f3["Check_revisit_date_blank"] = dynamic_source_merge_f3["RWM Revisit Date"].isna()

In [82]:
dynamic_source_merge_f3["Check_revisit_date_blank"] = dynamic_source_merge_f3[
    "Check_revisit_date_blank"].map({True:np.nan, False:"VALID"})

In [83]:
dynamic_source_merge_f4 = dynamic_source_merge_f3[['Primary_key', 'Check_revisit_date_blank']]

In [84]:
dynamic_source_merge2 = dynamic_source_merge.merge(dynamic_source_merge_f4, on='Primary_key', how='left')

In [85]:
dynamic_source_merge2['Validity'].fillna(dynamic_source_merge2['Check_revisit_date_blank'], inplace=True)

In [86]:
dynamic_source_merge3 = dynamic_source_merge2.merge(mid_match_f, on='s no', how='left')

In [87]:
dynamic_source_merge3['Objectives'].fillna(dynamic_source_merge3['Objectives_2'], inplace=True)
dynamic_source_merge3['Pincode'].fillna(dynamic_source_merge3['Pincode_2'], inplace=True)
dynamic_source_merge3['Phone Number'].fillna(dynamic_source_merge3['Phone Number_2'], inplace=True)
dynamic_source_merge3['City'].fillna(dynamic_source_merge3['City_2'], inplace=True)
dynamic_source_merge3['Cluster'].fillna(dynamic_source_merge3['Cluster_2'], inplace=True)

In [88]:
dynamic_source_merge_f = dynamic_source_merge3[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date',
                                              'RWM Remarks', 'Ingestion Date', 'RWM Revisit Status', 
                                               'RWM Revisit Date', 'Revisit Expire Date', 'Validity',
                                              'Pincode', 'Phone Number', 'City', 'Cluster', 
                                               'Failed Upload RWM MID','csv Google MID_x','Google csv Visit Date',
                                              'csv Validity', 's no']].rename(
    columns={'csv Google MID_x':'csv Google MID'})

In [89]:
dynamic_source_merge_f2 = dynamic_source_merge_f.sort_values(by="s no")

In [90]:
dynamic_source_merge_f2.to_csv(output_loc+"Dynamic_source_match.csv", index=False)

In [91]:
dynamic_source_merge_f.shape

(1991413, 19)

In [92]:
dynamic_source_merge_f2 = dynamic_source_merge_f[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date',
                                                 'RWM Remarks','Ingestion Date','Revisit Expire Date', 's no']]

In [93]:
dynamic_source_merge_f2 = dynamic_source_merge_f2[dynamic_source_merge_f2["RWM Remarks"].isin(['Success'])]

In [94]:
dynamic_source_merge_f2.drop_duplicates(subset=['MerchantId'], keep='last', inplace=True)

In [95]:
#dynamic_source_merge_f2[dynamic_source_merge_f2[
#    'RWM Remarks'].isin(['Success'])].to_csv(output_loc+'Dynamic_for_matching.csv', index=False)

In [96]:
dynamic_source_merge_f2.to_csv(output_loc+'Dynamic_for_matching.csv', index=False)

# Split file Based on Report Date

In [97]:
dynamic_source_merge_f['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022'],
      dtype=object)

In [98]:
file_no = 1

In [99]:
for i in date_list:
    dynamic_source_merge_f[dynamic_source_merge_f[
        'Report Date'].isin(i)].to_csv(output_loc+"Dynamic Source Split/"+str(file_no)+'.csv', index=False)
    file_no += 1

In [100]:
input_loc_s = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 4 Matching revisit with the dynamic Source/Result/Dynamic Source Split/"

In [101]:
sort_3 = pd.read_csv(input_loc_s+"3.csv")

In [102]:
sort_3["s no"].replace({",":""}, regex=True, inplace=True)

In [103]:
sort_3["s no"] = sort_3["s no"].astype(int)

In [104]:
sort_3.sort_values(by='s no', inplace=True)

In [105]:
sort_3.to_csv(input_loc_s+"3.csv", index=False)

In [106]:
file_size = 200000
batch_no_3 = 1

In [107]:
for i in pd.read_csv(input_loc_s+"3.csv", chunksize=file_size):
    i.to_csv(input_loc_s+'3-'+str(batch_no_3)+'.csv', index=False)
    batch_no_3 += 1

In [ ]:
Stop =  

# Extra